In [3]:
import pandas as pd
import seaborn as sns
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestRegressor
df_882 = pd.read_csv('D:\\2_ASW\OneDrive - Asesoftware S.A.S\\Escritorio\\reutilizacion_06_bioactivity_data_3class_pIC50_pubchem_fp_882.csv')
df_882.shape

(7049, 882)

In [4]:
X_882 = df_882.drop('pIC50', axis=1)
Y_882 = df_882.pIC50

In [3]:
#dividir dataset en entrenamiento y prueba
X_train_882, X_test_882, Y_train_882, Y_test_882 = train_test_split(X_882, Y_882, test_size=0.2,random_state=10)
X_train_882.shape, Y_train_882.shape, X_test_882.shape, Y_test_882.shape

((5639, 881), (5639,), (1410, 881), (1410,))

In [6]:
#Elimi

In [7]:
#Modelo de regresion lineal
import numpy as np
model_RFR = RandomForestRegressor(n_estimators=102, max_depth=16, random_state=0, criterion='squared_error',  min_samples_split=2)
model_RFR.fit(X_train_882, Y_train_882)
r2 = model_RFR.score(X_test_882, Y_test_882)
r2

0.6515353011702275

In [8]:
from sklearn.tree import DecisionTreeRegressor 
  
model_DTR_882 = DecisionTreeRegressor(random_state=6, max_depth=8, max_leaf_nodes=100, min_samples_leaf=2, splitter='random')
model_DTR_882.fit(X_train_882, Y_train_882)
predictions = model_DTR_882.predict(X_test_882)

#r2 = model1.score(X_test, Y_test)

score = model_DTR_882.score(X_test_882, Y_test_882)
print(score)

0.43344799332994555


In [9]:
from sklearn.ensemble import ExtraTreesRegressor
ETR=ExtraTreesRegressor(n_estimators=100, random_state=0)
ETR_model=ETR.fit(X_train_882, Y_train_882)

In [10]:
Y_pred_882 =ETR.predict(X_test_882)

In [11]:
#sacar media de Y_pred_882
media = Y_pred_882.mean()
media

media = 7

#calcular mediana
mediana = np.median(Y_pred_882)
mediana

4.826662975097878

In [12]:
Y_train_882_1 = [1 if i >= mediana else 0 for i in Y_train_882]

In [13]:
Y_test_882_1 = [1 if i >= mediana else 0 for i in Y_test_882]

In [14]:
#Importar el modelo
import lightgbm as ltb
#Importar las métricas del modelo 
from sklearn import metrics
#se define el modelo
LGBM_model= ltb.LGBMClassifier()

## LGBM

In [15]:
#Ajuste del modelo con los datos de entrenamiento
LGBM_fit=LGBM_model.fit(X_train_882,Y_train_882_1)

In [16]:
#Hacer predicciones con el modelo (Varible respuesta sólo 0 o 1)
LGBM_pred=LGBM_model.predict(X_test_882)
print(LGBM_pred)

[0 0 1 ... 0 1 0]


In [17]:
#Hacer predicciones con el modelo (Probabilidades)
LGBM_pred_prob=LGBM_model.predict_proba(X_test_882)

In [18]:
#Matriz de confusión
#Se va a contruir la matriz de confusión  para el modelo 
from sklearn.metrics import confusion_matrix
confusion_matrix_LGBM = confusion_matrix(Y_test_882_1, LGBM_pred)
Conf_matrix_LGBM=print(confusion_matrix_LGBM)

cm_display = metrics.ConfusionMatrixDisplay(confusion_matrix = Conf_matrix_LGBM, display_labels=[0,1],)
#mimportar plot
import matplotlib.pyplot as plt

[[600  95]
 [140 575]]


In [19]:
#Métricas del modelo 
from sklearn.metrics import classification_report
nombres=['0','1']
#Métricas del modelo
print(classification_report(Y_test_882_1, LGBM_pred,target_names=nombres))

              precision    recall  f1-score   support

           0       0.81      0.86      0.84       695
           1       0.86      0.80      0.83       715

    accuracy                           0.83      1410
   macro avg       0.83      0.83      0.83      1410
weighted avg       0.83      0.83      0.83      1410



In [20]:
#Curva ROC del modelo
from sklearn import svm, datasets
from sklearn.metrics import roc_curve, auc
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import label_binarize
from sklearn.multiclass import OneVsRestClassifier
from sklearn.metrics import roc_auc_score

#Definir la función
def plot_roc_curve(fper, tper):
    plt.plot(fper, tper, color='blue', label='ROC')
    plt.plot([0, 1], [0, 1], color='green', linestyle='--')
    plt.xlabel('Tasa Falsos Positivos')
    plt.ylabel('Tasa Verdaderos Positivos')
    plt.title('Curva característica de funcionamiento del receptor (ROC)')
    plt.legend()
    plt.show()

#Curva ROC sin reducción de dimensionalidad
fper, tper, thresholds = roc_curve(Y_test_882_1,LGBM_pred)
plot_roc_curve(fper, tper)
#mostrar el área bajo la curva
auc_score = roc_auc_score(Y_test_882_1,LGBM_pred)
print(auc_score)

0.8337525783568949


## **Modelo XGBClassifier**

In [21]:
#Importar el modelo de clasificación
from xgboost import XGBClassifier

In [22]:
#Ajustar el modelo de clasificación
xgbc = XGBClassifier()

In [23]:
#Ajustar el modelo de clasificación
xgbc = XGBClassifier()
XGBC_fit=xgbc.fit(X_train_882,Y_train_882_1)

XGBC_pred=xgbc.predict(X_test_882)
print(XGBC_pred)
len(XGBC_pred)

[0 0 1 ... 0 1 0]


1410

In [24]:
#Matriz de confusión
#Se va a contruir la matriz de confusión  para el modelo 
from sklearn.metrics import confusion_matrix
confusion_matrix_XGCB= confusion_matrix(Y_test_882_1,XGBC_pred)
Conf_matrix_XGCB=print(confusion_matrix_XGCB)

[[596  99]
 [134 581]]


In [25]:
#Métricas del modelo 
from sklearn.metrics import classification_report
nombres=['0','1']
#Métricas del modelo
print(classification_report(Y_test_882_1, XGBC_pred,target_names=nombres))

              precision    recall  f1-score   support

           0       0.82      0.86      0.84       695
           1       0.85      0.81      0.83       715

    accuracy                           0.83      1410
   macro avg       0.84      0.84      0.83      1410
weighted avg       0.84      0.83      0.83      1410



In [26]:
#calcular la precisión del modelo
from sklearn.metrics import accuracy_score
accuracy_score(Y_test_882_1,XGBC_pred)

0.8347517730496454

## 165 variables

In [94]:
#eliminar variables con varianzas bajas
from sklearn.feature_selection import VarianceThreshold
selection = VarianceThreshold(threshold=(.8 * (1 - .8)))    
X_882_165 = selection.fit_transform(X_882)
#resultado
X_882_165.shape #Se eliminan 717 variables

(7049, 165)

In [95]:
#dividir dataset en entrenamiento y prueba
X_train_882_165, X_test_882_165, Y_train_882_165, Y_test_882_165 = train_test_split(X_882_165, Y_882, test_size=0.2,random_state=10)

In [96]:
#Modelo de regresion lineal
import numpy as np
model_RFR = RandomForestRegressor(n_estimators=80, max_depth=17, random_state=0, criterion='squared_error',  min_samples_split=2)
model_RFR.fit(X_train_882_165, Y_train_882_165)
r2 = model_RFR.score(X_test_882_165, Y_test_882_165)
r2

0.6272573317937875

In [97]:
from sklearn.tree import DecisionTreeRegressor 
  
model_DTR_882_165 = DecisionTreeRegressor(random_state=2, max_depth=10, max_leaf_nodes=90, min_samples_leaf=2, splitter='random')
model_DTR_882_165.fit(X_train_882_165, Y_train_882_165)
predictions = model_DTR_882_165.predict(X_test_882_165)

score = model_DTR_882_165.score(X_test_882_165, Y_test_882_165)
print(score)

0.44319647748047886


In [ ]:
#Matriz de confusión de forma gráfica
import matplotlib.pyplot as plt
import numpy
from sklearn import metrics
Conf_matrix_LGBM= metrics.confusion_matrix(Yc1_test,LGBM_pred)

cm_display = metrics.ConfusionMatrixDisplay(confusion_matrix = Conf_matrix_LGBM, display_labels=[0,1],)

cm_display.plot()